In [1]:
import os 
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from einops.layers.torch import Rearrange
from tqdm.notebook import tqdm
from termcolor import cprint

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [3]:
class ThingsEEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str):
        super().__init__()
        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        
        self.X = torch.from_numpy(np.load(f"data/{split}/eeg.npy")).to(torch.float32)
        self.subject_idxs = torch.from_numpy(np.load(f"data/{split}/subject_idxs.npy"))

        if split in ["train", "val"]:
            self.y = torch.from_numpy(np.load(f"data/{split}/labels.npy"))
        else:
            self.y = None # testセットにはラベルがない

        print(f"[{split.upper()} SET] EEG: {self.X.shape}, Subject Indices: {self.subject_idxs.shape}", end="")
        if self.y is not None:
            print(f", Labels: {self.y.shape}")
        else:
            print()


    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        if self.y is not None:
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_classes(self) -> int:
        return 5 # animal, food, clothing, tool, vehicle

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size: int = 3, p_drop: float = 0.1):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)
        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X_skip = X
            X = self.conv0(X)
            X = X + X_skip # スキップ接続
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))
        
        X_skip = X
        X = self.conv1(X)
        X = X + X_skip # スキップ接続
        X = F.gelu(self.batchnorm1(X))

        return self.dropout(X)


class BasicConvClassifier(nn.Module):
    def __init__(self, num_classes: int, seq_len: int, in_channels: int, hid_dim: int = 128):
        super().__init__()
        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ConvBlock(hid_dim, hid_dim),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, num_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.blocks(X)
        return self.head(X)

In [5]:
# ハイパーパラメータ
lr = 0.001
batch_size = 512
epochs = 20

# データローダー
train_set = ThingsEEGDataset("train")
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

val_set = ThingsEEGDataset("val")
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[TRAIN SET] EEG: torch.Size([118800, 17, 100]), Subject Indices: torch.Size([118800]), Labels: torch.Size([118800])
[VAL SET] EEG: torch.Size([59400, 17, 100]), Subject Indices: torch.Size([59400]), Labels: torch.Size([59400])


In [6]:
# モデルの初期化
model = BasicConvClassifier(
    num_classes=train_set.num_classes,
    seq_len=train_set.seq_len,
    in_channels=train_set.num_channels
).to(device)

# 評価指標（正解率）
def accuracy(y_pred, y):
    return (y_pred.argmax(dim=-1) == y).float().mean()

# 最適化手法
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# TensorBoardの準備
writer = SummaryWriter("runs/eeg_experiment_1")

In [7]:
max_val_acc = 0.0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # ------ 訓練 ------
    model.train()
    train_loss_list, train_acc_list = [], []
    for X, y, subject_idxs in tqdm(train_loader, desc="Train"):
        X, y = X.to(device), y.to(device)
        
        y_pred = model(X)
        loss = F.cross_entropy(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss_list.append(loss.item())
        train_acc_list.append(accuracy(y_pred, y).item())

    avg_train_loss = np.mean(train_loss_list)
    avg_train_acc = np.mean(train_acc_list)

    # ------ 検証 ------
    model.eval()
    val_loss_list, val_acc_list = [], []
    with torch.no_grad():
        for X, y, subject_idxs in tqdm(val_loader, desc="Validation"):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = F.cross_entropy(y_pred, y)
            val_loss_list.append(loss.item())
            val_acc_list.append(accuracy(y_pred, y).item())
            
    avg_val_loss = np.mean(val_loss_list)
    avg_val_acc = np.mean(val_acc_list)

    # --- ログ表示 & TensorBoard記録 ---
    print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
    print(f"  Val   Loss: {avg_val_loss:.4f}, Val   Acc: {avg_val_acc:.4f}")

    writer.add_scalar("Loss/train", avg_train_loss, epoch)
    writer.add_scalar("Accuracy/train", avg_train_acc, epoch)
    writer.add_scalar("Loss/val", avg_val_loss, epoch)
    writer.add_scalar("Accuracy/val", avg_val_acc, epoch)

    # --- モデルのベストパラメータを保存 ---
    if avg_val_acc > max_val_acc:
        cprint(f"  New best validation accuracy! Saving model to model_best.pt", "cyan")
        torch.save(model.state_dict(), "model_best.pt")
        max_val_acc = avg_val_acc

writer.close()
print("\nTraining finished.")

Epoch 1/20


Train:   0%|          | 0/233 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

  Train Loss: 1.4899, Train Acc: 0.3816
  Val   Loss: 1.4766, Val   Acc: 0.3924
  New best validation accuracy! Saving model to model_best.pt
Epoch 2/20


Train:   0%|          | 0/233 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

  Train Loss: 1.4353, Train Acc: 0.3984
  Val   Loss: 1.4893, Val   Acc: 0.3798
Epoch 20/20


Train:   0%|          | 0/233 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

  Train Loss: 1.4291, Train Acc: 0.4017
  Val   Loss: 1.4934, Val   Acc: 0.3754

Training finished.


In [8]:
test_set = ThingsEEGDataset("test")
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


model = BasicConvClassifier(
    num_classes=test_set.num_classes,
    seq_len=test_set.seq_len,
    in_channels=test_set.num_channels
).to(device)

model.load_state_dict(torch.load("model_best.pt", map_location=device))
print("Best model weights loaded.")

preds = []
model.eval()
with torch.no_grad(): 
    for X, subject_idxs in tqdm(test_loader, desc="Evaluation"):
        X = X.to(device)
        y_pred = model(X)
        preds.append(y_pred.detach().cpu()) # 予測結果をCPUに移してから保存

# 全ての予測結果を一つのテンサーに結合し、NumPy配列に変換
preds = torch.cat(preds, dim=0).numpy()

# 提出用ファイルとして保存
np.save("submission.npy", preds)
print(f"\nSubmission file 'submission.npy' saved with shape: {preds.shape}")

[TEST SET] EEG: torch.Size([59400, 17, 100]), Subject Indices: torch.Size([59400])
Best model weights loaded.


/var/folders/ps/pftl20sn6t563rd47ngx0_f40000gn/T/ipykernel_80322/3993394019.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_best

Evaluation:   0%|          | 0/117 [00:00<?, ?it/s]


Submission file 'submission.npy' saved with shape: (59400, 5)
